### Código LEC4 2024-2

Grupo A 

In [ ]:
# Importación de librerías:
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
import scipy.stats as stats
from scipy.stats import beta

##### 2.1 Estimación de Máxima Verosimilitud 

a) Generar conjunto de datos

In [ ]:
# Conjunto de datos: Media = 5, Desviación estandar = 2.5
data = np.random.normal(loc = 5, scale = 2.5, size = 1000)

- b) Función de verosimilitud: 
$$L(\mu, \sigma^2) = \prod_{i = 1}^{1000} \frac{1}{\sqrt{2 \pi \sigma^2}} e^{\left( -\frac{(x_i - \mu)^2}{2 \sigma^2} \right)}$$

Simplificando: 
$$\displaystyle L(\mu, \sigma^2) =  \left( \frac{1}{\sqrt{2 \pi \sigma^2}} \right)^{1000} e^{\left( -\frac{1}{2 \sigma^2}\sum_{i=1}^{1000}(x_i - \mu)^2 \right)}$$

- c) log-verisimilitud:
$$ \log(L(\mu, \sigma^2)) = -\frac{n}{2}\log(2\sqrt{\pi}\sigma^2) - \frac{1}{2\sigma^2}\sum_{i=1}^{1000}\left(x_i -\mu\right)^2$$


d) Estimar los parametros

In [ ]:
# Log-verisimilitud:
def log_veri(params):
    mu = params[0]
    sigma = params[1]
    n = len(data)
    L = -n/2 * np.log(2 * np.pi) - n/2 * np.log(sigma**2) - 1/(2 * sigma**2) * np.sum((data - mu)**2)
    return -L

In [ ]:
# Definimos params iniciales:
params = np.array([5, 2.5])

# Usamos minimize para estimar los parámetros:
resultados = minimize(log_veri, params)

# Imprimimos los resultados:
mu = resultados.x[0]
sigma = resultados.x[1]
print('Media estimada:', mu, '\nDesviación estandar estimada:', sigma)

e) Histograma con los parametros y data inicial

In [ ]:
# Histogramas
plt.figure(figsize=(10, 6))
plt.hist(data, bins=30, color='pink', edgecolor='black', alpha=0.7, density=True)
plt.title('Histograma de los datos iniciales y estimación de la distribución normal')
plt.xlabel('Valores')  
plt.ylabel('Densidad')

# Distribución original
x_original = np.linspace(params[0] - 3 * params[1], params[0] + 3 * params[1], 1000)
pdf_original = norm.pdf(x_original, params[0], params[1])
plt.plot(x_original, pdf_original, color='green', lw=2, label='Distribución Normal Original')

# Distribución estimada
x_estimado = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 1000)
pdf_estimado = norm.pdf(x_estimado, mu, sigma)
plt.plot(x_estimado, pdf_estimado, color='red', lw=2, label='Distribución Normal Estimada')

# Marcar la media estimada
plt.axvline(x=mu, color='red', linestyle='--', label='Media Estimada')

plt.legend()
plt.grid()
plt.show()


f) Calcular un intervalo de confianza de 95% de los parametros, luego graficar:
- El intervalo de confianza se calcula como $$IC = \mu \pm Z_{\alpha/2} \cdot SE$$, donde SE $$\frac{\sigma}{\sqrt{n}}$$

In [ ]:
# Calculamos SE:
SE_media = sigma / np.sqrt(1000)
z_critico = stats.norm.ppf(0.975)

margen1 = z_critico * SE_media
ICmedia = (mu - margen1, mu + margen1)

SE_std = sigma / np.sqrt(2 * (999))

margen2 = z_critico * SE_std
ICstd = (sigma - margen2, sigma + margen2)

print(f"Intervalo de confianza del 95% de la media: ({ICmedia[0]:.4f}, {ICmedia[1]:.4f})")
print(f"Intervalo de confianza del 95% de la desviación estándar: ({ICstd[0]:.4f}, {ICstd[1]:.4f})")


In [ ]:
# Gráficos
plt.figure(figsize=(12, 6))
plt.hist(data, bins=30, color='navy', edgecolor='black', alpha=0.7, density=True)
plt.title('Histograma de los datos con intervalos de confianza del 95%')
plt.xlabel('Valores')  
plt.ylabel('Densidad')

# Marcar la media
plt.axvline(x=mu, color='red', linestyle='--', label='Media Estimada')

# Marcar el intervalo de confianza para la media
plt.axvline(x=ICmedia[0], color='green', linestyle='--', label='Límite Inferior IC Media (95%)')
plt.axvline(x=ICmedia[1], color='orange', linestyle='--', label='Límite Superior IC Media (95%)')

# Marcar el intervalo de confianza para la desviación estándar 
plt.fill_betweenx(y=[0, 0.15], x1=ICstd[0], x2=ICstd[1], 
                  color='purple', alpha=0.2, label='Intervalo de Confianza STD')

# Añadir leyenda y cuadrícula
plt.legend()
plt.grid()
plt.show()

#### 2.2 Inferencia Bayesiana - MAP

a) Realice una serie de 500 lanzamientos de la moneda del stand. Plantee una función
y simule estos lanzamientos en Python. Use la inferencia bayesiana para actualizar
la estimación de θ después de cada lanzamiento. Explique su implementación de la
simulación y la inferencia.

In [ ]:
true_theta = 0.7

def simular_lanzamientos(n_lanzamientos, theta):
    return np.random.rand(n_lanzamientos) < theta

alpha_prior = 1
beta_prior = 1

n_lanzamientos = 500
resultados = simular_lanzamientos(n_lanzamientos, true_theta)

alpha_post = alpha_prior
beta_post = beta_prior
estimaciones_theta = []

for resultado in resultados:
    if resultado:
        alpha_post += 1
    else:
        beta_post += 1
    estimaciones_theta.append((alpha_post - 1) / (alpha_post + beta_post - 2))

plt.plot(estimaciones_theta, label="Estimación MAP de θ")
plt.axhline(y=true_theta, color='r', linestyle='--', label="Valor real de θ (0.7)")
plt.xlabel("Número de lanzamientos")
plt.ylabel("Estimación de θ")
plt.title("Actualización de la estimación de θ después de cada lanzamiento")
plt.legend()
plt.grid(True)
plt.show()

estimaciones_theta[-1]

b) Visualice cómo cambia la distribución posterior a medida que observas más lanzamientos. ¿Cómo afecta la cantidad de lanzamientos a tu certeza sobre si la moneda
está sesgada? Realice un gráfico para 1, 5, 10, 50, 100 y 500 lanzamientos.

In [ ]:
true_theta = 0.7

def simular_lanzamientos(n_lanzamientos, theta):
    return np.random.rand(n_lanzamientos) < theta

alpha_prior = 1
beta_prior = 1

n_lanzamientos = 500
resultados = simular_lanzamientos(n_lanzamientos, true_theta)

lanzamientos_a_graficar = [1, 5, 10, 50, 100, 500]
x = np.linspace(0, 1, 1000)

plt.figure(figsize=(15, 10))

for i, n in enumerate(lanzamientos_a_graficar):
    alpha_post_n = alpha_prior + np.sum(resultados[:n])
    beta_post_n = beta_prior + n - np.sum(resultados[:n])
    
    plt.subplot(2, 3, i + 1)
    plt.plot(x, beta.pdf(x, alpha_post_n, beta_post_n), label=f"{n} lanzamientos")
    plt.axvline(x=true_theta, color='r', linestyle='--', label="Valor real de θ")
    plt.title(f"Posterior después de {n} lanzamientos")
    plt.xlabel("θ")
    plt.ylabel("Densidad")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

c) Después de realizar todos los lanzamientos, use la distribución posterior para calcular
una estimación MAP de θ. ¿Qué valor de θ se obtiene como el más probable?

In [ ]:
# Cálculo de la moda después de todos los lanzamientos
theta_MAP = (alpha_post - 1) / (alpha_post + beta_post - 2)
theta_MAP


d) ¿Qué sucede si eliges una prior distinta, como una que favorezca valores de θ muy cercanos a 0.5, o una que considere que la moneda está sesgada para obtener cruz? ¿Cómo
cambia esto la evolución de la posterior y la estimación final? ¿Qué conclusiones obtiene frente al tamaño de la muestra de datos? Realice experimentos que respalden sus
conclusiones.

In [ ]:
# Parámetros reales de la moneda
true_theta = 0.7
n_lanzamientos = 500

# Experimentos con diferentes priors
priors = {
    "Uniforme (Beta(1,1))": (1, 1),
    "Cercano a 0.5 (Beta(20,20))": (20, 20),
    "Sesgado hacia cruz (Beta(1,10))": (1, 10)
}

resultados = simular_lanzamientos(n_lanzamientos, true_theta)

#Graficas para cada prior
plt.figure(figsize=(12, 8))
for nombre_prior, (alpha_prior, beta_prior) in priors.items():
    alpha_post = alpha_prior
    beta_post = beta_prior
    estimaciones_theta = []

    for resultado in resultados:
        if resultado:
            alpha_post += 1 
        else:
            beta_post += 1 
        theta_MAP = (alpha_post - 1) / (alpha_post + beta_post - 2)
        estimaciones_theta.append(theta_MAP)

    plt.plot(estimaciones_theta, label=f"{nombre_prior}: θ_MAP")
    
plt.axhline(y=true_theta, color='r', linestyle='--', label="Valor real de θ (0.7)")
plt.xlabel("Número de lanzamientos")
plt.ylabel("Estimación de θ")
plt.title("Evolución de la estimación de θ con diferentes priors")
plt.legend()
plt.grid(True)
plt.show()








e) Compara tu estimación MAP con la estimación de máxima verosimilitud (MLE). ¿Hay
alguna diferencia significativa entre ambas? ¿Qué factores podrían explicar estas diferencias?

Usando el ejemplo del apunte obtenemos que estimador de máxima
verosimilitud pˆ para una distribución de Bernoulli es:

$$\hat{p} = \frac{\sum_{i=1}^{n} X_i}{n}$$


In [ ]:
# Cálculo de MLE
theta_MLE = np.sum(resultados) / n_lanzamientos

# Cálculo de MAP
theta_MAP = (alpha_post - 1) / (alpha_post + beta_post - 2)

print(f"Estimación MLE de θ: {theta_MLE}")
print(f"Estimación MAP de θ: {theta_MAP}")